In [5]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split, LeaveOneOut, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('SamsungData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

In [19]:
from sklearn.pipeline import Pipeline


xgBoost_SearchSpace = {
    'booster': ['gblinear'],
    'n_estimators': [225],
    'learning_rate': [.225],
    'reg_lambda': [0, 0.05, .1],
    'reg_alpha': [0, 0.05, .1],
    'eval_metric': ['rmse', 'auc'],
    'feature_selector': ['cyclic', 'random', 'greedy', 'thrifty'],
    'objective': ['reg:squaredlogerror'],
    'updater': ['shotgun', 'coord_descent'],
    'top_k': [0, 1 , 2, 3]            
}

rf_SearchSpace = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Splitting criterion
    'splitter': ['best', 'random'],               # Strategy to choose split at each node
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],      # Maximum depth of the tree
    'min_samples_split': [2, 5, 10, 15, 20],              # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 8, 16],                # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None], # Number of features to consider for the best split
    'max_leaf_nodes': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]        # Maximum number of leaf nodes in the tree
}

svr_SearchSpace = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

lasso_SearchSpace = {
    'alpha': [.1, .125, .175, .2, .15, .18, .17, .165, .185],  # Different values for alpha
    'tol': [.0001],  # Different values for tolerance
    'max_iter': [2000],  # Different values for maximum iterations
    'selection': ['cyclic']
}


scoring = {
    'r2': 'r2',  
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

"""
lbfgs search
    'hidden_layer_sizes': [(10, 20, 10)],  # Adjusting number of neurons and layers
    'activation': ['identity'],  # Activation functions 
    'solver': ['lbfgs'],  # Optimization algorithms
    'batch_size': [25],  # Batch size
    'max_iter': [2500],  # Maximum number of iterations
    'random_state': [42],

    'tol': [1e-4, 1e-5, 1e-6, 1e-3, 1e-2, 1e-1],
    'alpha': [0.001],
    'early_stopping': [True, False],
    'validation_fraction': [],
    'max_fun': []
"""

searchSpace = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Splitting criterion
    'max_depth': [None, 2, 5, 7, 10, 20, 40],      # Maximum depth of the tree
           # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None], # Number of features to consider for the best split
    'n_estimators': [5, 10, 20, 200, 100],
    'random_state': [1]
}


In [20]:

GS = GridSearchCV(estimator=RandomForestRegressor(),
                  param_grid= searchSpace,
                  scoring = 'neg_mean_absolute_error',
                  refit='neg_mean_absolute_error',
                  cv=5,
                  verbose=4)

In [21]:
GS.fit(X, y)

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
[CV 1/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=5, random_state=1;, score=nan total time=   0.0s
[CV 2/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=5, random_state=1;, score=nan total time=   0.0s
[CV 3/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=5, random_state=1;, score=nan total time=   0.0s
[CV 4/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=5, random_state=1;, score=nan total time=   0.0s
[CV 5/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=5, random_state=1;, score=nan total time=   0.0s
[CV 1/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=10, random_state=1;, score=nan total time=   0.0s
[CV 2/5] END criterion=mse, max_depth=None, max_features=auto, n_estimators=10, random_state=1;, score=nan total time=   0.0s
[CV 3/5] END criterion=mse, max_depth=None, max_features=au

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
1575 fits failed out of a total of 2100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
700 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae'],
                         'max_depth': [None, 2, 5, 7, 10, 20, 40],
                         'max_features': ['auto', 'sqrt', 'log2', None],
                         'n_estimators': [5, 10, 20, 200, 100],
                         'random_state': [1]},
             refit='neg_mean_absolute_error', scoring='neg_mean_absolute_error',
             verbose=4)

In [23]:
GS.best_params_

{'criterion': 'friedman_mse',
 'max_depth': 5,
 'max_features': None,
 'n_estimators': 10,
 'random_state': 1}

: 

In [13]:
GS.best_score_

-9.560424630986999

In [14]:
GS.best_estimator_

RandomForestRegressor(criterion='friedman_mse', max_depth=20,
                      max_features='log2', max_leaf_nodes=5)